In [1]:

# use at last

def average_point(sorted_p):
    distance = 10
    # sort column 0 then column 1
    # idex = np.lexsort([ori_point[:, 1], ori_point[:, 0]])

    # print(sorted_p)

    feature_list = np.empty([0, 2])
    feature_list = np.vstack([feature_list, sorted_p[0]])  # total 48 feature from 0-47
    # feature_list numpy
    feature_data = 0
    tf = False
    data_list = []
    for i in range(len(sorted_p)):
        reference = sorted_p[i]
        for i in range(len(feature_list)):
            if np.linalg.norm(feature_list[i] - reference) < distance:
                # np.array_equal(feature_list[i],reference):
                tf = True
                feature_data = i
                break
            tf = False
            feature_data = i + 1

        if tf == True:
            data_list.append(feature_data)
        else:
            feature_list = np.vstack([feature_list, reference])
            data_list.append(feature_data)
    data_list = np.array(data_list).reshape([-1, 1])
    sorted_p = np.hstack((sorted_p, data_list))

    position = []
    for i in range(48):
        position.append(np.mean(sorted_p[np.where(sorted_p[:, 2] == i)], axis=0))
    position = np.array(position)
    position = position[:, [0, 1]]

    return (position)


# part 1.5
# check cross color
def filter_p(ori_point, I_bin, width, height):
    new_point = []
    detect_size = 3

    for i in range(len(ori_point)):
        x = ori_point[i, 1]
        y = ori_point[i, 0]
        x = int(x)
        y = int(y)
        if x in range(detect_size, width - detect_size) and y in range(detect_size, height - detect_size):
            p_1 = I_bin[y - detect_size, x - detect_size]
            p_2 = I_bin[y - detect_size, x + detect_size]
            p_3 = I_bin[y + detect_size, x + detect_size]
            p_4 = I_bin[y + detect_size, x - detect_size]
            p_5 = I_bin[y - detect_size, x]
            p_6 = I_bin[y, x + detect_size]
            p_7 = I_bin[y + detect_size, x]
            p_8 = I_bin[y, x - detect_size]
            tf_1 = ((p_1 == p_3) and (p_2 == p_4) and (p_2 * p_4) != (p_1 * p_3))
            tf_2 = ((p_5 == p_7) and (p_6 == p_8) and (p_5 * p_7) != (p_6 * p_8))

            if tf_1 or tf_2:
                new_point.append(ori_point[i])

    new_point = np.array(new_point)

    return new_point


def saddle(filter_point, I):
    ###implement saddle point
    # part 2
    # build a small patch for each point
    patch_size = 20  # need to be even
    sdpoint_arr = np.empty((0, 2))
    for i in range(len(filter_point)):
        # saddle point position
        sdpoint_x = filter_point[i][1]
        sdpoint_y = filter_point[i][0]

        #     a1=sdpoint_x-patch_size/2
        #     a2=sdpoint_x+patch_size/2
        #     a3=sdpoint_y-patch_size/2
        #     a4=sdpoint_y+patch_size/2
        #     print(a1,a2,a3,a4)
        #  do the saddle point based on I_binary
        patch = I[int(sdpoint_y - patch_size / 2):int(sdpoint_y + patch_size / 2),
                int(sdpoint_x - patch_size / 2):int(sdpoint_x + patch_size / 2)]
        m, n = patch.shape
        # clear B
        B = []
        # x2 xy y2 x y 1 (m,n) begin from 0,0
        A = np.empty((0, 6))
        for x in range(0, n):
            for y in range(0, m):
                A = np.vstack((A, [x ** 2, x * y, y ** 2, x, y, 1.]))
                B.append(patch[y, x])
        # w=(a,b,c,d,e,f) for parameters
        w = np.linalg.lstsq(A, B, rcond=None)[0]
        (a, b, c, d, e, f) = w
        # solve the intersection
        # [2a b],[b 2c]
        C = np.array([[float(2 * a), float(b)], [float(b), float(2 * c)]])
        D = np.array([-d, -e])
        pt = np.linalg.pinv(C).dot(D)
        # correct the position of pt
        # print(pt.shape)
        pt = pt - [patch_size / 2, patch_size / 2] + [sdpoint_x, sdpoint_y]
        pt = pt.reshape((1, 2))
        # print(pt)(x,y)
        sdpoint_arr = np.append(sdpoint_arr, pt, axis=0)

        # print(sdpoint_arr.shape) #(n,2)
    sdpoint_arr = sdpoint_arr[:, [1, 0]]
    return sdpoint_arr


def boundary_delete(array, bpoly):
    point_arr = np.empty((0, 2))
    x = np.array(bpoly[0, :])
    y = np.array(bpoly[1, :])
    # print(len(point_arr)
    for i in range(len(array)):
        x_point = array[i, 1]
        y_point = array[i, 0]
        y_1 = (y[1] - y[0]) / (x[1] - x[0]) * (x_point - x[1]) + y[1] + 5  # y_1 should be smaller than y_point
        y_3 = (y[3] - y[2]) / (x[3] - x[2]) * (x_point - x[3]) + y[3] - 5
        x_2 = (y_point - y[2]) / ((y[2] - y[1]) / (x[2] - x[1])) + x[2] - 5
        x_4 = (y_point - y[0]) / ((y[0] - y[3]) / (x[0] - x[3])) + x[0] + 5

        #         y_1 = (y[1, 1] - y[1, 0]) / (x[0, 1] - x[0, 0]) * (x_point - x[0, 1]) + y[1, 1]  # y_1 should be smaller than y_point
        #         y_3 = (y[1, 3] - y[1, 2]) / (x[0, 3] - x[0, 2]) * (x_point - x[0, 3]) + y[1, 3]
        #         x_2 = (y_point - y[1, 2]) / ((y[1, 2] - y[1, 1]) / (x[0, 2] - x[0, 1])) + x[0, 2]
        #         x_4 = (y_point - y[1, 0]) / ((y[1, 0] - y[1, 3]) / (x[0, 0] - x[0, 3])) + x[0, 0]

        if (y_point > y_1 and y_point < y_3 and x_point < x_2 and x_point > x_4):
            point_arr = np.vstack((point_arr, array[i]))

    return point_arr


from scipy.ndimage.filters import *


def cross_junctions(I, bounds, Wpts):
    # part one
    # change to binary
    I_binary = np.where(I > 115, 0, 1)
    # gaussian_filter filter
    I_gaussian = gaussian_filter(I, sigma=1)

    I_copy = I.copy()

    # harris corner
    # calculate gradient

    Ix, Iy = np.gradient(I_binary)
    Ix = gaussian_filter(Ix, sigma=1)
    Iy = gaussian_filter(Iy, sigma=1)

    # 2d array
    Ixx = Ix ** 2
    Ixy = Iy * Ix
    Iyy = Iy ** 2
    # Ixx=gaussian_filter(Ixx, sigma=2)
    # Ixy=gaussian_filter(Ixy, sigma=2)
    # Iyy=gaussian_filter(Iyy, sigma=2)
    height = I_binary.shape[0]  # 480
    width = I_binary.shape[1]  # 640

    k = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])

    Sxx = convolve(Ixx, k, mode='nearest')
    Sxy = convolve(Ixy, k, mode='nearest')
    Syy = convolve(Iyy, k, mode='nearest')

    det = (Sxx * Syy) - (Sxy ** 2)
    trace = Sxx + Syy
    r = det - 0.04 * (trace ** 2)

    # array_np = np.array(r)
    low_values_flags = r < 0  # Where values are low
    r[low_values_flags] = 0

    # max=np.amax(r)
    # print(max)

    # print(array_np)(y,x)
    point_array = []
    ## show the plot
    for i in range(r.shape[0]):
        for j in range(r.shape[1]):
            if r[i, j] != 0:
                # I_copy[i,j]=255
                point_array.append([i, j])
    # point_array is integer listxxxx turn to be a numpy array
    point_array = np.array(point_array)

    # print(point_array)
    # plt.plot(point_array[:,1],point_array[:,0],'ro', color='r')

    # filter point (y,x)
    # filter_point=filter_p(point_array,I_binary,width,height)

    # run the function
    # sdpoint(y,x)

    sdpoint = np.empty((0, 2))

    sdpoint = saddle(point_array, I_gaussian)

    sdpoint = boundary_delete(sdpoint, bounds)
    sdpoint = filter_p(sdpoint, I_binary, width, height)

   
    # np.set_printoptions(threshold=np.inf)
    # sdpoint

    # need to sort point !!
    # print(sorted_point)

    final_result = average_point(sdpoint)
    #   Ipts  - 2xn np.array of cross-junctions (x, y), relative to the upper
    #   left corner of I. These should be floating-point values.
    # $
    idex = np.lexsort([final_result[:, 1], final_result[:, 0]])
    
    final_result = final_result[idex, :]
    
    order=np.array([])
    for i in range(6):
        ordd=np.argsort(final_result[i*8:(i+1)*8,1])  
        ordd=ordd+8*i
        order=np.append(order,ordd)

    final_result=final_result[order.astype(int)]
    
    
    
    
    

    plt.imshow(I)
    plt.plot(final_result[:,1],final_result[:,0],'ro', color='r')



    Ipts = np.zeros((2, 48))
    Ipts = final_result.transpose()[[1, 0]]

    # ------------------
    return Ipts


In [2]:


import numpy as np
from imageio import imread
from mat4py import loadmat
import matplotlib.pyplot as plt


# Load the boundary.
#$
#bpoly = np.array([[514.0,1176,834,214],[10 ,398 ,820 ,558]])
bpoly = np.array([[160 ,410, 588, 345], [251, 2 ,182 ,420]])
#bpoly = np.array([[222, 503 ,501, 220],[173 ,175 ,383, 386]])



# Load the world points.
#bpoly = np.array(loadmat("bboxes.mat")["bpolyh1"])
Wpts = np.array(loadmat("world_pts.mat")["world_pts"])
SP = np.array(loadmat("saddle_points.mat")["Ipts"])
# Load the example target image.
#I = imread("image_000350.png")
I=np.loadtxt("Testcase2.txt")
Ipts = cross_junctions(I, bpoly, Wpts)

# # You can plot the points to check!
print(Ipts)




ModuleNotFoundError: No module named 'mat4py'

In [121]:
A=Ipts[[1,0]]

In [122]:
B=A.transpose()

In [50]:
#B y,x

array([[206.01853913, 263.41006699],
       [206.41720668, 292.59396625],
       [206.74225587, 321.55761819],
       [207.15078282, 350.41811021],
       [207.28884618, 379.16328646],
       [207.80304169, 435.9522276 ],
       [207.81684278, 407.69369153],
       [208.29144699, 463.98280861],
       [235.18569158, 263.18107921],
       [235.27507501, 292.35562211],
       [235.49097566, 321.40272431],
       [235.69212504, 350.20205579],
       [236.16871542, 378.93451154],
       [236.17120952, 407.32807486],
       [236.45798019, 435.53843694],
       [236.53817378, 463.56507783],
       [264.35922522, 262.88044743],
       [264.52823   , 292.22153218],
       [264.53604034, 321.00159751],
       [264.75533131, 378.53454257],
       [264.76652129, 350.19241789],
       [264.8870319 , 435.2370842 ],
       [265.01695709, 407.1063241 ],
       [265.2030172 , 463.24235741],
       [293.21432815, 262.67703863],
       [293.31587836, 291.88752814],
       [293.3278146 , 349.55163997],
 

In [125]:
bpoly

array([[160, 410, 588, 345],
       [251,   2, 182, 420]])

In [127]:
bpoly[0,0]  #x0

160

In [129]:
bpoly[1,0]#y0#

410

In [ ]:
bpoly[0,1]

In [134]:
np.arctan(-(bpoly[1,1]-bpoly[1,0])/(bpoly[0,1]-bpoly[0,0]))

0.7833941580641407

In [5]:
np.cos(0.7)

0.7648421872844885

In [120]:
order=np.array([])
for i in range(6):
    ordd=np.argsort(B[i*8:(i+1)*8,1])
    ordd=ordd+8*i
    order=np.append(order,ordd)

B[order.astype(int)]



array([[115.23206578, 349.22196419],
       [ 88.69293173, 374.61427302],
       [114.48566502, 400.2035698 ],
       [ 61.86372894, 400.48674492],
       [140.04862542, 425.38780086],
       [ 88.00387813, 425.78780057],
       [113.87206616, 451.09652487],
       [139.22571531, 475.67676873],
       [167.52325079, 299.05549818],
       [141.51653713, 323.97370155],
       [166.68640527, 349.83838629],
       [140.7195795 , 374.82591978],
       [165.93171964, 400.11921545],
       [165.11022658, 450.28731749],
       [164.30527819, 500.30237651],
       [189.14436148, 523.88020362],
       [193.29858318, 274.25787314],
       [192.41198983, 324.8165174 ],
       [191.59783962, 375.08067908],
       [216.04168922, 399.93260972],
       [190.72401374, 424.97622956],
       [215.23987754, 449.50494921],
       [190.00793978, 474.59071648],
       [214.29339947, 498.7270758 ],
       [218.70428301, 249.92008305],
       [242.98764042, 275.67474251],
       [217.79291892, 300.04850718],
 